In [ ]:
!pip install tensorpack
!pip install python-doctr==0.9.0
!pip install deepdoctection

In [ ]:
import deepdoctection as dd
from IPython.core.display import HTML
from matplotlib import pyplot as plt

# Instantiate the built-in analyzer
analyzer = dd.get_dd_analyzer()

# Analyze your bank statement PDF
df = analyzer.analyze(path="/content/Revoult Bank statement.pdf")
df.reset_state()  # Initialize the pipeline

# Process the document
doc = iter(df)
page = next(doc)

# Visualize the detected layout and tables
image = page.viz(show_figures=True, show_residual_layouts=True)
plt.figure(figsize=(25, 17))
plt.axis('off')
plt.imshow(image)

# Extract table data as HTML
if page.tables:
    HTML(page.tables[0].html)

# Extract all text content
print(page.text)


In [ ]:
# Process all tables in the document
for i, table in enumerate(page.tables):
    print(f"Table {i+1}:")
    print(HTML(table.html))
    print("\n" + "="*50 + "\n")


In [ ]:
import deepdoctection as dd
import pandas as pd
from IPython.display import HTML, display
import matplotlib.pyplot as plt

def extract_tables_deepdoctection(pdf_path):
    """
    Extract tables from bank statement using deepdoctection
    """
    # Initialize analyzer
    analyzer = dd.get_dd_analyzer()

    # Analyze document
    df = analyzer.analyze(path=pdf_path)
    df.reset_state()

    all_tables = []

    # Process each page
    page_num = 1
    for page in df:
        print(f"\n=== PAGE {page_num} ===")

        # Check if tables exist
        if page.tables:
            print(f"Found {len(page.tables)} table(s) on page {page_num}")

            for i, table in enumerate(page.tables):
                print(f"\nTable {i+1} on Page {page_num}:")

                # Method 1: Get HTML format
                try:
                    html_table = table.html
                    print("HTML Table:")
                    display(HTML(html_table))
                except:
                    print("HTML extraction failed")

                # Method 2: Extract structured data
                try:
                    table_data = []
                    for row in table.rows:
                        row_data = []
                        for cell in row.cells:
                            row_data.append(cell.text.strip())
                        table_data.append(row_data)

                    if table_data and len(table_data) > 1:
                        # Create DataFrame
                        df_table = pd.DataFrame(table_data[1:], columns=table_data[0])
                        print("\nStructured Table:")
                        print(df_table)

                        # Save to CSV
                        csv_filename = f"bank_table_page{page_num}_table{i+1}.csv"
                        df_table.to_csv(csv_filename, index=False)
                        print(f"Saved to: {csv_filename}")

                        all_tables.append({
                            'page': page_num,
                            'table_num': i+1,
                            'dataframe': df_table,
                            'html': html_table,
                            'filename': csv_filename
                        })

                except Exception as e:
                    print(f"Structured extraction failed: {e}")

                print("-" * 50)
        else:
            print(f"No tables found on page {page_num}")

        page_num += 1

    return all_tables

# CHANGE THIS PATH TO YOUR BANK STATEMENT PDF
pdf_path = "/content/Revoult Bank statement.pdf"  # <-- CHANGE THIS PATH

# Extract tables
extracted_tables = extract_tables_deepdoctection(pdf_path)

# Summary
print(f"\n=== SUMMARY ===")
print(f"Total tables extracted: {len(extracted_tables)}")
for table_info in extracted_tables:
    print(f"Page {table_info['page']}, Table {table_info['table_num']}: {table_info['filename']}")


In [ ]:
import deepdoctection as dd
from IPython.core.display import HTML
from matplotlib import pyplot as plt

# Instantiate the built-in analyzer
analyzer = dd.get_dd_analyzer()

# Analyze your bank statement PDF
df = analyzer.analyze(path="/content/Barclays_uk_bank_statement.pdf")
df.reset_state()  # Initialize the pipeline

# Process the document
doc = iter(df)
page = next(doc)

# Visualize the detected layout and tables
image = page.viz(show_figures=True, show_residual_layouts=True)
plt.figure(figsize=(25, 17))
plt.axis('off')
plt.imshow(image)

# Extract table data as HTML
if page.tables:
    HTML(page.tables[0].html)

# Extract all text content
print(page.text)


# for json or table 

In [ ]:
import tabula
import pandas as pd

# Read tables from PDF
try:
    tables = tabula.read_pdf('/content/Business Bank statment.pdf', pages='all')

    # Check if any tables were found
    if len(tables) > 0:
        print(f"Found {len(tables)} table(s)")

        # Process first table
        df = tables[0]

        # Export to JSON and Excel
        df.to_json('bank_data.json', orient='records', indent=2)
        df.to_excel('bank_data.xlsx', index=False)
        print("Data exported successfully!")
    else:
        print("No tables found in the PDF")

except Exception as e:
    print(f"Error reading PDF: {e}")


# trying deepdoctection with easy ocr

In [ ]:
# Install required packages
!pip install easyocr pdf2image pandas openpyxl fuzzywuzzy python-levenshtein

import easyocr
import pdf2image
import pandas as pd
import re
from fuzzywuzzy import fuzz
import numpy as np

class DynamicBankStatementExtractor:
    def __init__(self):
        self.reader = easyocr.Reader(['en'])

        # Common column headers across different banks
        self.common_headers = {
            'date': ['date', 'transaction date', 'value date', 'posting date', 'trans date'],
            'description': ['description', 'details', 'transaction details', 'reference', 'narrative', 'particulars'],
            'debit': ['debit', 'withdrawal', 'amount dr', 'debits', 'outgoing'],
            'credit': ['credit', 'deposit', 'amount cr', 'credits', 'incoming'],
            'amount': ['amount', 'transaction amount', 'value'],
            'balance': ['balance', 'running balance', 'available balance', 'closing balance'],
            'reference': ['reference', 'ref', 'ref no', 'transaction id', 'cheque no'],
            'category': ['category', 'type', 'transaction type']
        }

    def extract_bank_data_dynamic(self, pdf_path):
        """Extract bank data with dynamic column detection"""

        # Convert PDF to images
        images = pdf2image.convert_from_path(pdf_path)

        all_text_data = []
        detected_headers = []

        for i, image in enumerate(images):
            # Convert PIL Image to numpy array
            image_np = np.array(image)

            # Perform OCR
            results = self.reader.readtext(image_np)

            # Extract text with coordinates
            text_with_coords = []
            for (bbox, text, confidence) in results:
                if confidence > 0.5:
                    # Get center coordinates
                    x_center = (bbox[0][0] + bbox[2][0]) / 2
                    y_center = (bbox[0][1] + bbox[2][1]) / 2
                    text_with_coords.append({
                        'text': text.strip(),
                        'x': x_center,
                        'y': y_center,
                        'confidence': confidence
                    })

            all_text_data.extend(text_with_coords)

            # Detect headers from first page
            if i == 0:
                detected_headers = self.detect_column_headers(text_with_coords)

        print(f" Detected Headers: {detected_headers}")

        # Extract transactions
        transactions = self.extract_transactions_dynamic(all_text_data, detected_headers)

        # Create DataFrame
        if transactions:
            df = pd.DataFrame(transactions)

            # Clean and standardize data
            df = self.clean_dataframe(df)

            # Export
            df.to_excel('bank_transactions_dynamic.xlsx', index=False)
            df.to_json('bank_transactions_dynamic.json', orient='records', indent=2)

            return df
        else:
            print(" No transactions found")
            return pd.DataFrame()

    def detect_column_headers(self, text_data):
        """Dynamically detect column headers"""

        potential_headers = []

        # Look for text that might be headers
        for item in text_data:
            text = item['text'].lower()

            # Check if text matches any common header patterns
            for standard_name, variants in self.common_headers.items():
                for variant in variants:
                    similarity = fuzz.ratio(text, variant)
                    if similarity > 70:  # 70% similarity threshold
                        potential_headers.append({
                            'original': item['text'],
                            'standard': standard_name,
                            'x': item['x'],
                            'y': item['y'],
                            'similarity': similarity
                        })
                        break

        # Sort by Y coordinate (top to bottom) and X coordinate (left to right)
        potential_headers.sort(key=lambda x: (x['y'], x['x']))

        # Get unique headers (avoid duplicates)
        unique_headers = []
        used_standards = set()

        for header in potential_headers:
            if header['standard'] not in used_standards:
                unique_headers.append(header)
                used_standards.add(header['standard'])

        # Sort by X coordinate for final column order
        unique_headers.sort(key=lambda x: x['x'])

        return unique_headers

    def extract_transactions_dynamic(self, text_data, headers):
        """Extract transactions using dynamic headers"""

        if not headers:
            print("No headers detected, using fallback extraction")
            return self.fallback_extraction(text_data)

        transactions = []

        # Group text by rows (similar Y coordinates)
        rows = self.group_text_by_rows(text_data)

        for row in rows:
            if self.is_likely_transaction_row(row, headers):
                transaction = self.parse_transaction_row(row, headers)
                if transaction:
                    transactions.append(transaction)

        return transactions

    def group_text_by_rows(self, text_data, y_tolerance=20):
        """Group text items that are likely in the same row"""

        # Sort by Y coordinate
        sorted_data = sorted(text_data, key=lambda x: x['y'])

        rows = []
        current_row = []
        current_y = None

        for item in sorted_data:
            if current_y is None or abs(item['y'] - current_y) <= y_tolerance:
                current_row.append(item)
                current_y = item['y'] if current_y is None else current_y
            else:
                if current_row:
                    # Sort row items by X coordinate (left to right)
                    current_row.sort(key=lambda x: x['x'])
                    rows.append(current_row)
                current_row = [item]
                current_y = item['y']

        # Add the last row
        if current_row:
            current_row.sort(key=lambda x: x['x'])
            rows.append(current_row)

        return rows

    def is_likely_transaction_row(self, row, headers):
        """Determine if a row contains transaction data"""

        row_text = ' '.join([item['text'] for item in row])

        # Look for date patterns
        date_patterns = [
            r'\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}',
            r'\d{2}[\/\-]\d{2}[\/\-]\d{4}',
            r'\d{1,2}\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)',
        ]

        has_date = any(re.search(pattern, row_text) for pattern in date_patterns)

        # Look for amount patterns
        amount_patterns = [
            r'[\£\$\€\₹]?\s*\d+[,.]?\d*\.?\d{2}',
            r'\d+[,.]?\d*\.?\d{2}\s*[\£\$\€\₹]?'
        ]

        has_amount = any(re.search(pattern, row_text) for pattern in amount_patterns)

        return has_date or has_amount

    def parse_transaction_row(self, row, headers):
        """Parse a transaction row based on detected headers"""

        transaction = {}

        # Initialize all detected columns
        for header in headers:
            transaction[header['standard']] = ''

        # Map row items to columns based on X coordinates
        header_x_coords = [(h['x'], h['standard']) for h in headers]
        header_x_coords.sort()

        for item in row:
            # Find the closest header column
            closest_header = None
            min_distance = float('inf')

            for header_x, header_name in header_x_coords:
                distance = abs(item['x'] - header_x)
                if distance < min_distance:
                    min_distance = distance
                    closest_header = header_name

            if closest_header and min_distance < 100:  # Within reasonable distance
                # Append to existing value (in case multiple items map to same column)
                if transaction[closest_header]:
                    transaction[closest_header] += ' ' + item['text']
                else:
                    transaction[closest_header] = item['text']

        # Clean up the transaction data
        transaction = self.clean_transaction_data(transaction)

        # Only return if we have essential data
        if any(transaction.values()):
            return transaction

        return None

    def fallback_extraction(self, text_data):
        """Fallback extraction when headers can't be detected"""

        print("🔄 Using fallback extraction method")

        transactions = []
        rows = self.group_text_by_rows(text_data)

        for row in rows:
            row_text = ' '.join([item['text'] for item in row])

            # Use regex patterns to extract data
            transaction = self.extract_with_patterns(row_text)
            if transaction:
                transactions.append(transaction)

        return transactions

    def extract_with_patterns(self, text):
        """Extract transaction data using regex patterns"""

        transaction = {}

        # Date extraction
        date_match = re.search(r'\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}', text)
        if date_match:
            transaction['date'] = date_match.group()

        # Amount extraction
        amount_matches = re.findall(r'[\£\$\€\₹]?\s*\d+[,.]?\d*\.?\d{2}', text)
        if amount_matches:
            # First amount might be transaction amount, last might be balance
            transaction['amount'] = amount_matches[0].strip()
            if len(amount_matches) > 1:
                transaction['balance'] = amount_matches[-1].strip()

        # Description (remaining text after removing dates and amounts)
        clean_text = re.sub(r'\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4}', '', clean_text) # Corrected: used clean_text instead of text
        clean_text = re.sub(r'[\£\$\€\₹]?\s*\d+[,.]?\d*\.?\d{2}', '', clean_text) # Corrected: used clean_text instead of text
        transaction['description'] = clean_text.strip()

        return transaction if any(transaction.values()) else None

    def clean_transaction_data(self, transaction):
        """Clean and standardize transaction data"""

        cleaned = {}

        for key, value in transaction.items():
            if value:
                # Remove extra spaces and clean text
                cleaned_value = ' '.join(value.split())

                # Specific cleaning for different column types
                if key == 'date':
                    # Standardize date format
                    cleaned_value = self.standardize_date(cleaned_value)
                elif key in ['amount', 'debit', 'credit', 'balance']:
                    # Clean amount format
                    cleaned_value = self.clean_amount(cleaned_value)

                cleaned[key] = cleaned_value
            else:
                cleaned[key] = ''

        return cleaned

    def standardize_date(self, date_str):
        """Standardize date format"""
        # Try to parse and reformat date
        import datetime

        date_patterns = [
            ('%d/%m/%Y', r'\d{2}/\d{2}/\d{4}'),
            ('%d-%m-%Y', r'\d{2}-\d{2}-\d{4}'),
            ('%d/%m/%y', r'\d{2}/\d{2}/\d{2}'),
            ('%d-%m-%y', r'\d{2}-\d{2}-\d{2}'),
        ]

        for format_str, pattern in date_patterns:
            if re.match(pattern, date_str):
                try:
                    date_obj = datetime.datetime.strptime(date_str, format_str)
                    return date_obj.strftime('%Y-%m-%d')
                except:
                    continue

        return date_str  # Return original if can't parse

    def clean_amount(self, amount_str):
        """Clean and standardize amount format"""
        # Remove currency symbols and extra spaces
        cleaned = re.sub(r'[^\d.,\-]', '', amount_str)
        return cleaned

    def clean_dataframe(self, df):
        """Final DataFrame cleaning"""

        # Remove completely empty rows
        df = df.dropna(how='all')

        # Sort by date if date column exists
        date_columns = [col for col in df.columns if 'date' in col.lower()]
        if date_columns:
            try:
                df[date_columns[0]] = pd.to_datetime(df[date_columns[0]], errors='coerce')
                df = df.sort_values(date_columns[0])
            except:
                pass

        return df

# Usage
extractor = DynamicBankStatementExtractor()
df = extractor.extract_bank_data_dynamic('/content/Revoult Bank statement.pdf')

print("Extracted Data Preview:")
print(df.head())
print(f"\notal Transactions: {len(df)}")
print(f"Detected Columns: {list(df.columns)}")

In [ ]:
!apt-get update
!apt-get install poppler-utils